In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tcn import TCN
from kerastuner.tuners import RandomSearch

# ------------------ Utility Functions ------------------ #
def load_and_preprocess_data(filepath):
    """Load and preprocess the dataset."""
    data = pd.read_csv(filepath)
    data['date'] = pd.to_datetime(data['date'])
    df_grouped = data.groupby(['date', 'generation_type'], as_index=False)['megawatt_hours'].sum()
    return df_grouped

def standardize_generation_types(df, unique_types):
    """Ensure all generation types are represented for every date."""
    all_dates = df['date'].unique()
    standardized_rows = [
        {'date': date, 'generation_type': g_type, 'megawatt_hours': 0}
        for date in all_dates
        for g_type in unique_types
        if g_type not in df[df['date'] == date]['generation_type'].values
    ]
    return pd.concat([df, pd.DataFrame(standardized_rows)], ignore_index=True)

def scale_data(df):
    """Scale the data using MinMaxScaler."""
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    return scaled_data, scaler

def create_sequences(data, n_steps):
    """Create input-output sequences for time-series data."""
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)

# ------------------ Model Building Function ------------------ #
def build_tcn_model(hp):
    """Build the TCN model with hyperparameters."""
    dilation_depth = hp.Int('dilation_depth', min_value=2, max_value=6, step=1)
    dilations = [2**i for i in range(dilation_depth)]
    
    model = Sequential([
        Input(shape=(n_steps, X_train.shape[2])),
        TCN(
            nb_filters=hp.Int('nb_filters', min_value=32, max_value=256, step=32),
            kernel_size=hp.Choice('kernel_size', values=[2, 3, 5, 7]),
            nb_stacks=hp.Int('nb_stacks', min_value=1, max_value=3),
            dilations=dilations,
            padding='causal',  # Use causal padding to prevent data leakage
            use_skip_connections=hp.Boolean('use_skip_connections'),
            dropout_rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1),
            activation=hp.Choice('activation', values=['relu', 'tanh', 'selu', 'gelu']),
            use_batch_norm=hp.Boolean('use_batch_norm'),
            use_layer_norm=hp.Boolean('use_layer_norm'),
        ),
        Dense(
    units=X_train.shape[2],  # Match the output shape to the target
    activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'selu'])
)
    ])
    
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    if optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae', 'mape']
    )
    return model

# ------------------ Main Script ------------------ #
if __name__ == "__main__":
    # Load and preprocess data
    filepath = r'data/processed/canada_energy.csv'
    df_grouped = load_and_preprocess_data(filepath)
    unique_generation_types = df_grouped['generation_type'].unique()
    df_standardized = standardize_generation_types(df_grouped, unique_generation_types)
    df_pivot = df_standardized.pivot(index='date', columns='generation_type', values='megawatt_hours').fillna(0)
    df_pivot[df_pivot < 0] = 0

    # Scale data
    data_scaled, scaler = scale_data(df_pivot)

    # Define time steps and create sequences
    n_steps = 36
    X, y = create_sequences(data_scaled, n_steps)
    test_size = 24  # Assuming 24 months (2 years) for testing
    train_data, test_data = data_scaled[:-test_size], data_scaled[-test_size - n_steps:]
    X_train, y_train = create_sequences(train_data, n_steps)
    X_test, y_test = create_sequences(test_data, n_steps)

    # Set up hyperparameter tuning
    tuner = RandomSearch(
        build_tcn_model,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        directory='tcn_hyperparam_search',
        project_name='energy_forecasting_tcn'
    )

    # Run hyperparameter search
    tuner.search(
        X_train, y_train,
        epochs=50,
        validation_data=(X_test, y_test),
        batch_size=32,
        verbose=2
    )

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Print optimal hyperparameters
    print(f"""
    Optimal hyperparameters:
    - Filters: {best_hps.get('nb_filters')}
    - Kernel size: {best_hps.get('kernel_size')}
    - Stacks: {best_hps.get('nb_stacks')}
    - Dilation depth: {best_hps.get('dilation_depth')}
    - Dropout rate: {best_hps.get('dropout_rate')}
    - Activation: {best_hps.get('activation')}
    - Batch normalization: {best_hps.get('use_batch_norm')}
    - Layer normalization: {best_hps.get('use_layer_norm')}
    - Dense units: {best_hps.get('dense_units')}
    - Dense activation: {best_hps.get('dense_activation')}
    - Optimizer: {best_hps.get('optimizer')}
    - Learning rate: {best_hps.get('learning_rate')}
    """)

    # Retrain the model with the best hyperparameters
    best_model = tuner.hypermodel.build(best_hps)
    history = best_model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=2
    )

    # Plot training and validation loss
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title("Model Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()


Trial 2 Complete [00h 00m 02s]

Best val_loss So Far: None
Total elapsed time: 00h 00m 03s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
6                 |3                 |dilation_depth
256               |224               |nb_filters
5                 |7                 |kernel_size
3                 |2                 |nb_stacks
False             |False             |use_skip_connections
0.3               |0.3               |dropout_rate
relu              |relu              |activation
True              |False             |use_batch_norm
False             |False             |use_layer_norm
64                |16                |dense_units
selu              |relu              |dense_activation
adam              |sgd               |optimizer
0.0068984         |0.00012798        |learning_rate

Epoch 1/50


Traceback (most recent call last):
  File "c:\Users\matth\anaconda3\envs\AISE4010\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\matth\anaconda3\envs\AISE4010\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\matth\anaconda3\envs\AISE4010\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\matth\anaconda3\envs\AISE4010\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\matth\anaconda3\envs\AISE4010\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\matth\anaconda3\envs\AISE4010\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\matth\anaconda3\envs\AISE4010\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\matth\anaconda3\envs\AISE4010\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermod

In [ ]:
#print the best hyperparameters
print(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)


{'dilation_depth': 6, 'nb_filters': 80, 'kernel_size': 3, 'nb_stacks': 1, 'use_skip_connections': False, 'dropout_rate': 0.0, 'activation': 'tanh', 'optimizer': 'adam'}
